# 1. Document Loading

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## PDFs

In [9]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/Yetiskin2020.pdf")
pages = loader.load()

In [3]:
len(pages)

18

In [4]:
page = pages[0]

In [5]:
print(page.page_content[0:500])

Full Terms & Conditions of access and use can be found at
https://www.tandfonline.com/action/journalInformation?journalCode=hppc20
Popular Communication
The International Journal of Media and Culture
ISSN: (Print) (Online) Journal homepage: https://www.tandfonline.com/loi/hppc20
Paratactic commoning: collective knowledge
production networking as political struggle
Ebru Yetiskin
To cite this article: Ebru Yetiskin (2020): Paratactic commoning: collective knowledge production
networking as politic


In [10]:
page.metadata

{'producer': 'iText 4.2.0 by 1T3XT',
 'creator': 'Arbortext Advanced Print Publisher 11.0.3433/W Unicode',
 'creationdate': '2020-06-25T12:52:06+05:30',
 'keywords': 'Commons; commoning; data; control; tactic; paratactic; obfuscation',
 'moddate': '2020-06-30T04:01:46-07:00',
 'source': 'docs/Yetiskin2020.pdf',
 'total_pages': 18,
 'page': 0,
 'page_label': '1'}

## Youtube

In [6]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.blob_loaders import FileSystemBlobLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [ ]:
url="https://www.youtube.com/watch?v=vtLfCO4IGXY&ab_channel=DiEM25"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),  # fetch from youtube
    #FileSystemBlobLoader(save_dir, glob="*.m4a"),   #fetch locally
    OpenAIWhisperParser()
)
docs = loader.load()

In [25]:
docs[0].page_content[0:500]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nhandbook/titles-for-programmers.md at master · basecamp/handbook · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\nGitHub Copilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nSecurity\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nActions\n        Automa'

## URLs

In [22]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/titles-for-programmers.md")

In [23]:
docs = loader.load()

In [ ]:
print(docs[0].page_content[:500])

# 2. Document Splitting

![Alt text](images/splitters.png)

### Key Differences

| Feature                           | `RecursiveCharacterTextSplitter`              | `CharacterTextSplitter`         |
|-----------------------------------|--------------------------------|-------------------------|
| **Splitting Strategy**           | Tries to break at meaningful places | Splits at a fixed character (e.g., space) |
| **Handles Word Boundaries?**      | ✅ Yes                          | ❌ No (may cut words)  |
| **Performance**                   | Slightly slower but better structured chunks | Fast but less optimal chunks |
| **Best for**                      | Long documents with structured text (e.g., articles, books) | Simple text that doesn’t require structure |


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [4]:
chunk_size =26
chunk_overlap = 4

In [12]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator=' '
)

In [ ]:
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

In [31]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2)

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [32]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [36]:
c_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Recursive splitting details

`RecursiveCharacterTextSplitter` is recommended for generic text. 

In [37]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [38]:
len(some_text)

496

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

In [40]:
c_splitter.split_text(some_text)

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [41]:
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [42]:
#Let's reduce the chunk size a bit and add a period to our separators:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
r_splitter.split_text(some_text)

In [43]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/NG2023.pdf")
pages = loader.load()

In [46]:
len(pages)

41

In [44]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [47]:
docs = text_splitter.split_documents(pages)
len(docs)

80

## Token splitting

We can also split on token count explicity, if we want.

This can be useful because LLMs often have context windows designated in tokens.

Tokens are often ~4 characters.

In [48]:
from langchain.text_splitter import TokenTextSplitter

In [50]:
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [51]:
text1 = "foo bar bazzyfoo"
text_splitter.split_text(text1)

['foo', ' bar', ' b', 'az', 'zy', 'foo']

In [52]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [53]:
docs[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.0 (Macintosh)', 'creationdate': '2022-12-13T16:08:00-05:00', 'moddate': '2022-12-13T16:08:04-05:00', 'trapped': '/False', 'source': 'docs/NG2023.pdf', 'total_pages': 41, 'page': 0, 'page_label': '1'}, page_content='PAGE 1\nFounder, DeepLearning.')

In [54]:
docs[5]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.0 (Macintosh)', 'creationdate': '2022-12-13T16:08:00-05:00', 'moddate': '2022-12-13T16:08:04-05:00', 'trapped': '/False', 'source': 'docs/NG2023.pdf', 'total_pages': 41, 'page': 1, 'page_label': '2'}, page_content='\nelectricity. It will \ntransform and')

In [55]:
pages[0].metadata

{'producer': 'Adobe PDF Library 17.0',
 'creator': 'Adobe InDesign 18.0 (Macintosh)',
 'creationdate': '2022-12-13T16:08:00-05:00',
 'moddate': '2022-12-13T16:08:04-05:00',
 'trapped': '/False',
 'source': 'docs/NG2023.pdf',
 'total_pages': 41,
 'page': 0,
 'page_label': '1'}

In [56]:
pages[10].metadata

{'producer': 'Adobe PDF Library 17.0',
 'creator': 'Adobe InDesign 18.0 (Macintosh)',
 'creationdate': '2022-12-13T16:08:00-05:00',
 'moddate': '2022-12-13T16:08:04-05:00',
 'trapped': '/False',
 'source': 'docs/NG2023.pdf',
 'total_pages': 41,
 'page': 10,
 'page_label': '11'}

## Context aware splitting

Chunking aims to keep text with common context together.

A text splitting often uses sentences or other delimiters to keep related text together but many documents (such as Markdown) have structure (headers) that can be explicitly used in splitting.

We can use `MarkdownHeaderTextSplitter` to preserve header metadata in our chunks, as show below.

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [ ]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n
## Chapter 2\n\n \
Hi this is Molly"""

In [59]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [60]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [61]:
md_header_splits[0]

Document(metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1'}, page_content='Hi this is Jim  \nHi this is Joe')

In [62]:
md_header_splits[1]

Document(metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1', 'Header 3': 'Section'}, page_content='Hi this is Lance')

# Vectorstores and Embeddings

![overview.png](images/overview.png)

![](images/vectorstore.png)

In [2]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("docs/Geron2017.pdf"),
    PyPDFLoader("docs/NG2018.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
print(f"Total documents loaded: {len(docs)}")
print(f"First document: {docs[0] if docs else 'No documents loaded'}")

Total documents loaded: 682
First document: page_content='Aurélien Géron
Hands-On  
Machine Learning  
with Scikit-Learn  
& TensorFlow  
CONCEPTS, TOOLS, AND TECHNIQUES  
TO BUILD INTELLIGENT SYSTEMS
 D o w n l o a d   f r o m   f i n e l y b o o k   w w w . f i n e l y b o o k . c o m' metadata={'producer': 'Antenna House PDF Output Library 6.2.609 (Linux64)', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'creationdate': '2017-03-10T21:55:34+00:00', 'author': 'Aurélien Géron', 'moddate': '2017-05-16T09:54:54+08:00', 'title': 'Hands-On Machine Learning with Scikit-Learn and TensorFlow', 'trapped': '/False', 'source': 'docs/Geron2017.pdf', 'total_pages': 564, 'page': 0, 'page_label': 'Cover'}


In [4]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [5]:
splits = text_splitter.split_documents(docs)

In [6]:
print(f"Total Chunks: {len(splits)}")
print(f"First Chunk: {splits[0].page_content}")  # Show first chunk

Total Chunks: 1211
First Chunk: Aurélien Géron
Hands-On  
Machine Learning  
with Scikit-Learn  
& TensorFlow  
CONCEPTS, TOOLS, AND TECHNIQUES  
TO BUILD INTELLIGENT SYSTEMS
 D o w n l o a d   f r o m   f i n e l y b o o k   w w w . f i n e l y b o o k . c o m


In [7]:
print(f"Type of first element in splits: {type(splits[0])}")

Type of first element in splits: <class 'langchain_core.documents.base.Document'>


## Embeddings

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_17578/1333021179.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


In [20]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [21]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [22]:
embedding1

[-0.027551146146196668,
 -0.005382069836976813,
 -0.02582130760099973,
 -0.03305632611697975,
 -0.027349121791237472,
 0.022639416094150544,
 -0.010435849398653005,
 -0.008125188737472597,
 0.0025079497065223677,
 -0.019646922070829638,
 0.000673547850878873,
 0.02924310407710086,
 -0.00539785296060417,
 0.0007114275338490437,
 0.0002138622943082356,
 0.0140786105368597,
 0.029874432747485417,
 -0.001194787906827953,
 0.004210956382759252,
 -0.0038700394688583597,
 -0.011692191441061522,
 0.006875160130779538,
 0.012992726602155665,
 -0.04724857570274996,
 -0.0022980333240882078,
 0.004706548979229199,
 0.016932211768408267,
 -0.00013178968182733469,
 -0.02588444009550916,
 -0.016351390583747363,
 0.02669254124063623,
 0.0029972288672768573,
 -0.01551803755840361,
 -0.024760676477834954,
 0.006035494135381614,
 -0.014949843245173624,
 0.009785581352444648,
 -0.011515419897641582,
 -0.004349848708870307,
 -0.01087777872286414,
 -0.017058478620072263,
 0.011180816186625503,
 0.0069382926

In [9]:
import numpy as np

In [10]:
np.dot(embedding1, embedding2)

0.9631510802407727

In [11]:
np.dot(embedding1, embedding3)

0.7702031204123162

In [12]:
np.dot(embedding2, embedding3)

0.759053971445477

## Vectorstores

| **Vector DB**  | **Best For**                      | **Persistence**  | **Metadata Filtering** | **Cloud / Local** |
|---------------|--------------------------------|---------------|------------------|----------------|
| **Chroma**   | RAG, fast prototyping        | ✅ Yes       | ✅ Yes          | Local / Cloud  |
| **FAISS**    | Large-scale local search     | ❌ No (manual) | ❌ No          | Local         |
| **Pinecone** | Cloud-scale search          | ✅ Yes (Cloud) | ✅ Yes          | Cloud         |
| **Weaviate** | Hybrid search (text + vector) | ✅ Yes       | ✅ Yes          | Local / Cloud  |
| **Milvus**   | Distributed vector search    | ✅ Yes       | ✅ Yes          | Local / Cloud  |


In [8]:
from langchain.vectorstores import Chroma

In [9]:
persist_directory = 'docs/chroma/'

In [10]:
!rm -rf ./docs/chroma  # remove old database files if any

In [11]:
# Remove empty text chunks
documents = [doc for doc in splits if doc.page_content.strip()]
# Remove exact duplicate chunks
unique_documents = list({doc.page_content: doc for doc in documents}.values())

In [12]:
print(f"Total valid documents: {len(documents)}")
print(f"First document content: {documents[0].page_content[:200]}")  # Print first 200 characters
print(f"First document metadata: {documents[0].metadata}")


Total valid documents: 1211
First document content: Aurélien Géron
Hands-On  
Machine Learning  
with Scikit-Learn  
& TensorFlow  
CONCEPTS, TOOLS, AND TECHNIQUES  
TO BUILD INTELLIGENT SYSTEMS
 D o w n l o a d   f r o m   f i n e l y b o o k   w w w 
First document metadata: {'producer': 'Antenna House PDF Output Library 6.2.609 (Linux64)', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'creationdate': '2017-03-10T21:55:34+00:00', 'author': 'Aurélien Géron', 'moddate': '2017-05-16T09:54:54+08:00', 'title': 'Hands-On Machine Learning with Scikit-Learn and TensorFlow', 'trapped': '/False', 'source': 'docs/Geron2017.pdf', 'total_pages': 564, 'page': 0, 'page_label': 'Cover'}


In [13]:
# Remove Surrogate Unicode Characters
import unicodedata
from langchain.schema import Document

def clean_text(text):
    return unicodedata.normalize("NFKD", text).encode("utf-8", "ignore").decode("utf-8")

documents = [
    Document(page_content=clean_text(doc.page_content), metadata=doc.metadata)
    for doc in documents
]

In [14]:

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory
)

# try persist_directory=None if you get tenant error. It can also work in-memory mode.

In [15]:
print(vectordb._collection.count())  #tells us how many document chunks have been embedded and indexed in Chroma
print(vectordb.similarity_search("What is AI?", k=3))  # Test retrieval

1211
[Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 1, 'page_label': '2', 'producer': 'Skia/PDF m71', 'source': 'docs/NG2018.pdf', 'total_pages': 118}, page_content='Machine Learning Yearning is a\n \n \ndeeplearning.ai project.\n \n \n \n \n \n \n \n \n \n \n \n© 2018 Andrew Ng. All Rights Reserved.\n \n \n \n \nPage 2\nMachine Learning Yearning-Draft\nAndrew Ng'), Document(metadata={'author': 'Aurélien Géron', 'creationdate': '2017-03-10T21:55:34+00:00', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'moddate': '2017-05-16T09:54:54+08:00', 'page': 25, 'page_label': '4', 'producer': 'Antenna House PDF Output Library 6.2.609 (Linux64)', 'source': 'docs/Geron2017.pdf', 'title': 'Hands-On Machine Learning with Scikit-Learn and TensorFlow', 'total_pages': 564, 'trapped': '/False'}, page_content='If you already know all the Machine Learning basics, you may want\nto skip directly to Chapter 2 . If you are not sure, tr\ny to answe

In [16]:
docs_ss= vectordb.similarity_search("What are neural networks?", k=3)
print(docs_ss)


[Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 91, 'page_label': '92', 'producer': 'Skia/PDF m71', 'source': 'docs/NG2018.pdf', 'total_pages': 118}, page_content='Neural networks are commonly used in end-to-end learning systems. The term “end-to-end”\n \nrefers to the fact that we are asking the learning algorithm to go directly from the input to\n \nthe desired output. I.e., the learning algorithm directly connects the “input end” of the\n \nsystem to the “output end.”\n \nIn problems where data is abundant, end-to-end systems have been remarkably successful.\n \nBut they are not always a good choice. The next few chapters will give more examples of\n \nend-to-end systems as well as give advice on when you should and should not use them.\n \n \n \n \n \n \n \n \n \n \n \nPage 92\nMachine Learning Yearning-Draft\nAndrew Ng'), Document(metadata={'author': 'Aurélien Géron', 'creationdate': '2017-03-10T21:55:34+00:00', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64

In [17]:
docs_ss[0].page_content

'Neural networks are commonly used in end-to-end learning systems. The term “end-to-end”\n \nrefers to the fact that we are asking the learning algorithm to go directly from the input to\n \nthe desired output. I.e., the learning algorithm directly connects the “input end” of the\n \nsystem to the “output end.”\n \nIn problems where data is abundant, end-to-end systems have been remarkably successful.\n \nBut they are not always a good choice. The next few chapters will give more examples of\n \nend-to-end systems as well as give advice on when you should and should not use them.\n \n \n \n \n \n \n \n \n \n \n \nPage 92\nMachine Learning Yearning-Draft\nAndrew Ng'

In [18]:
import textwrap


# Print formatted results
for i, doc in enumerate(docs_ss):
    print(f"\n🔹 **Result {i+1}:**")
    print(f"📄 Source: {doc.metadata.get('source', 'Unknown')}, Page: {doc.metadata.get('page', 'N/A')}")
    print("📝 Content:")
    print(textwrap.fill(doc.page_content, width=80))  # Wrap text to 80 characters
    print("-" * 100)  # Divider


🔹 **Result 1:**
📄 Source: docs/NG2018.pdf, Page: 91
📝 Content:
Neural networks are commonly used in end-to-end learning systems. The term “end-
to-end”   refers to the fact that we are asking the learning algorithm to go
directly from the input to   the desired output. I.e., the learning algorithm
directly connects the “input end” of the   system to the “output end.”   In
problems where data is abundant, end-to-end systems have been remarkably
successful.   But they are not always a good choice. The next few chapters will
give more examples of   end-to-end systems as well as give advice on when you
should and should not use them.                       Page 92 Machine Learning
Yearning-Draft Andrew Ng
----------------------------------------------------------------------------------------------------

🔹 **Result 2:**
📄 Source: docs/Geron2017.pdf, Page: 277
📝 Content:
4 In the context of Machine Learning, the phrase “neural networks” generally
refers to ANNs, not BNNs. 5 Drawing of a co

# Retrieval
Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow. 
Let's get our vectorDB from before.

![](images/MMR.png)

![](images/LLM_retrieval.png)

![](images/compression.png)

In [49]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [50]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [51]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [52]:
smalldb.similarity_search(question, k=2)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

## Addressing Diversity: Maximum marginal relevance

Last class we introduced one problem: how to enforce diversity in the search results.
 
`Maximum marginal relevance` strives to achieve both relevance to the query *and diversity* among the results.

In [53]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(metadata={}, page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [56]:
docs_mmr = vectordb.max_marginal_relevance_search("What are neural networks?", k=2, fetch_k=3)

In [57]:
docs_mmr[0].page_content

'Neural networks are commonly used in end-to-end learning systems. The term “end-to-end”\n \nrefers to the fact that we are asking the learning algorithm to go directly from the input to\n \nthe desired output. I.e., the learning algorithm directly connects the “input end” of the\n \nsystem to the “output end.”\n \nIn problems where data is abundant, end-to-end systems have been remarkably successful.\n \nBut they are not always a good choice. The next few chapters will give more examples of\n \nend-to-end systems as well as give advice on when you should and should not use them.\n \n \n \n \n \n \n \n \n \n \n \nPage 92\nMachine Learning Yearning-Draft\nAndrew Ng'

## Addressing Specificity: working with metadata

In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on `metadata`.

`metadata` provides context for each embedded chunk.

In [20]:
question = "what did they say about deep learning in NG book?"

In [21]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"docs/NG2018.pdf"}
)

In [60]:
for d in docs:
    print(d.metadata)

{'creationdate': '', 'creator': 'PyPDF', 'page': 54, 'page_label': '55', 'producer': 'Skia/PDF m71', 'source': 'docs/NG2018.pdf', 'total_pages': 118}
{'creationdate': '', 'creator': 'PyPDF', 'page': 47, 'page_label': '48', 'producer': 'Skia/PDF m71', 'source': 'docs/NG2018.pdf', 'total_pages': 118}
{'creationdate': '', 'creator': 'PyPDF', 'page': 8, 'page_label': '9', 'producer': 'Skia/PDF m71', 'source': 'docs/NG2018.pdf', 'total_pages': 118}


## Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [22]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [23]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/NG2018.pdf`, `docs/Zheng2018.pdf`, or `docs/Geron2017.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.self_query.chroma import ChromaTranslator


llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

document_content_description = "AI books"
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    structured_query_translator=ChromaTranslator(),
    verbose=True
)

/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_17578/1248247315.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)


In [34]:
question = "what did they say about artifical intelligence in the first page?"

In [35]:
docs = retriever.get_relevant_documents(question)

In [30]:
docs

[Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 68, 'page_label': '69', 'producer': 'Skia/PDF m71', 'source': 'docs/NG2018.pdf', 'total_pages': 118}, page_content='human-level performance.\n \n \nThere are many important machine learning applications where machines surpass human\n \nlevel performance. For example, machines are better at predicting movie ratings, how long it\n \ntakes for a delivery car to drive somewhere, or whether to approve loan applications. Only a\n \nsubset of techniques apply once humans have a hard time identifying examples that the\n \nalgorithm is clearly getting wrong. Consequently, progress is usually slower on problems\n \nwhere machines already surpass human-level performance, while progress is faster when\n \nmachines are still trying to catch up to humans.\n \n \n \n \n \nPage 69\nMachine Learning Yearning-Draft\nAndrew Ng'),
 Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 1, 'page_label': '2', 'producer': 'Skia/P

## Compression

In [38]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [39]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [40]:
# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
compressor = LLMChainExtractor.from_llm(llm)

In [41]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [42]:
question = "what did they say about ai?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

• There is now a huge quantity of data available to train neural networks, and
ANNs frequen tly outperform other ML techniques on ver
y large and complex
problems.
• The tremendous increase in computing power since the 1990s now makes it pos‐
sible to train large neural networks in a reasonable amount of time. This is in
part due to Moore
’s Law, but also thanks to the gaming industry, which has pro‐
duced powerful GPU cards by the millions.
• The training algorithms have been improved. To be fair they are only slightly dif‐
ferent from the ones used in the 1990s, but these relatively small tweaks ha
ve a
huge positive impact.
• Some theoretical limitations of ANNs have turned out to be benign in practice.
For example, man
y people thought that ANN training algorithms were doomed
because they were likely to get stuck in local optima, but it turns out that this is
rather rare in practice (or when it is the case, they are usually fairly close to the
global optimum).
• ANNs s

## Other types of Retrival

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [60]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [61]:
# Load PDF
loader = PyPDFLoader("docs/NG2023.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [63]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [64]:
question = "What are major topics for this book?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

Document(metadata={}, page_content='✓\n✓\nOvercoming Imposter Syndrome PAGE 40\nMake Every Day Count\nFinal Thoughts\nEvery year on my birthday, I get to thinking about the days behind and those \nthat may lie ahead.\nWhen I ask friends, many choose a number in the hundreds of \nthousands. (Many others can’t resist calculating the answer, \nto my annoyance!)\nWhen I was a grad student, I remember plugging my statistics \ninto a mortality calculator to figure out my life expectancy. \nThe calculator said I could expect to live a total of 27,649 \ndays. It struck me how small this number is. I printed it in a \nlarge font and pasted it on my office wall as a daily reminder.\nThat’s all the days we have to spend with loved ones, learn, \nbuild for the future, and help others. Whatever you’re doing \ntoday, is it worth 1/30,000 of your life?\nHow many days is a \ntypical human lifespan?\nMaybe you’re good at math; I’m sure you’ll be able to answer the following question \nvia a quick calcu

In [65]:
question = "what did they say about ethics"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(metadata={}, page_content='algorithms, deployment processes, and software stacks a particular company uses. You \nmay be surprised — if you’re not already familiar with the data-centric AI movement — to \nlearn how much time most machine learning engineers spend iteratively cleaning datasets.\n✓\n✓\nWhat do you do in a typical week or day?\nWhat are the most important tasks in this role?\nWhat skills are most important for success?\nHow does your team work together to accomplish its goals?\nWhat is the hiring process?\nConsidering candidates who stood out in the past, what enabled them to shine?\n✓\n✓\n✓\n✓\n✓\n✓\nUsing Informational Interviews to Find the Right Job CHAPTER 8 PAGE 30\nFinding someone to interview isn’t always easy, but many people who are in senior positions today \nreceived help when they were new from those who had entered the field ahead of them, and many \nare eager to pay it forward. If you can reach out to someone who’s already in your network — \nperhap

# Question Answering

In [66]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [67]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_68203/3010969844.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [68]:
print(vectordb._collection.count())

4815


In [69]:
question = "What are major topics for this book?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [70]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

## RetrievalQA chain

In [45]:
from langchain.chains import RetrievalQA

In [46]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [47]:
result = qa_chain({"query": question})

/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_17578/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [48]:
result["result"]

' They said that there is now a huge quantity of data available to train neural networks, and ANNs frequently outperform other ML techniques on very large and complex problems.'

## Promt

In [49]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [50]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [51]:
question = "Why to use neural networks?"

In [52]:
result = qa_chain({"query": question})

In [83]:
result["result"]

'Neural networks are best suited for end-to-end learning systems where the algorithm connects input directly to the desired output. They are successful in problems with abundant data but may not always be the best choice. Thanks for asking!'

In [84]:
result["source_documents"][0]

Document(metadata={'author': 'Aurélien Géron', 'creationdate': '2017-03-10T21:55:34+00:00', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'moddate': '2017-05-16T09:54:54+08:00', 'page': 17, 'page_label': 'xvi', 'producer': 'Antenna House PDF Output Library 6.2.609 (Linux64)', 'source': 'docs/Geron2017.pdf', 'title': 'Hands-On Machine Learning with Scikit-Learn and TensorFlow', 'total_pages': 564, 'trapped': '/False'}, page_content='Part II, Neural Networks and Deep Learning, covers the following topics:\n• Wha t are neural nets? Wha\nt are they good for?\n• Building and training neural nets using TensorFlow.\n• The most important neural net architectures: feedforward neural nets, convolu‐\ntional nets, recurrent nets, long short-term memory (LSTM) nets, and a\nutoen‐\ncoders.\n• Techniques for training deep neural nets.\n• Scaling neural networks for huge datasets.\n• Reinforcement learning.\nThe first part is based mostly on Scikit-Learn while

### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [86]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [87]:
question = "Why to use Neural Networks?"
result = qa_chain({"query": question})
result["result"]

'Neural networks are used because they can model complex functions efficiently, especially deep neural networks. They have a hierarchical architecture that allows them to automatically take advantage of the structured nature of real-world data. This hierarchical structure helps neural networks converge faster to good solutions and improves their ability to generalize to new datasets. Additionally, neural networks are commonly used in end-to-end learning systems, where the learning algorithm directly connects the input to the desired output, making them successful in problems with abundant data.'

In [88]:
question = "what kind of complex functions do you mean?"
result = qa_chain({"query": question})
result["result"]

'The complex functions referred to in the context are those where multiple features are combined together to create more intricate features. These complex features aim to capture essential information from raw data more effectively. The idea is that by creating more sophisticated input features, the model can be simpler, easier to train, evaluate, and make better predictions.'

# Chat - Memory

In [89]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_68203/1956280058.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


## ConversationalRetrievalChain

In [90]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [91]:
question = "Why to use Neural Networks?"
result = qa({"question": question})

In [92]:
result['answer']

'Neural networks are used because they can model complex functions efficiently, especially deep neural networks. They have a hierarchical architecture that allows them to automatically take advantage of the structured nature of real-world data. This hierarchical structure helps neural networks converge faster to a good solution and improves their ability to generalize to new datasets. Additionally, neural networks are commonly used in end-to-end learning systems, where the learning algorithm directly connects the input to the desired output, making them successful in problems with abundant data.'

In [97]:
question = "give examples to complex functions?"
result = qa({"question": question})

In [99]:
result['answer']

'Neural networks can efficiently model complex functions such as image captioning, machine translation, question answering, speech recognition, and text-to-speech synthesis. These tasks involve processing rich inputs like images, text, audio, and generating corresponding outputs, showcasing the capability of neural networks to handle diverse and complex functions.'